In [1]:
import xarray as xr 

ds = xr.open_dataset('ml-drought-forecasting/soil-water-forecasting/data/01_raw/ERA5_monthly_averaged_data_on_single_levels.nc')

In [2]:
ds

<xarray.Dataset> Size: 54GB
Dimensions:    (date: 562, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 4kB 19780101 19780201 19780301 ... 20240901 20241001
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 9kB ...
Data variables: (12/23)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB ...
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    swvl1      (date, latitude, longitude) float32 2GB ...
    cvh        (date, latitude, longitude) float32 2GB ...
    lai_hv     (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

# Format datatype

In [3]:
import pandas as pd

# 1. Convert 'date' coordinate to datetime
ds['date'] = pd.to_datetime(ds['date'].astype(str), format='%Y%m%d')

# Fill na values

In [4]:
# INTO 3.data_preprocessing.ipynb 

import xarray as xr
from typing import List, Union

def fillna_in_variables(
    ds: xr.Dataset,
    variables: List[str],
    fill_value: Union[int, float]
) -> xr.Dataset:
    """
    Fills NaN values in specified variables of an xarray.Dataset with a provided value.

    Args:
        ds (xr.Dataset): The input dataset containing the data variables.
        variables (List[str]): A list of variable names in the dataset for which to fill NaN values.
        fill_value (Union[int, float]): The value to fill NaN values with (e.g., 0).

    Returns:
        xr.Dataset: A new dataset with NaN values filled in the specified variables.
    
    Raises:
        ValueError: If a variable in `variables` does not exist in the dataset.
    
    Example:
        filled_ds = fillna_in_variables(ds, ["t2m", "swvl1"], fill_value=0)
    """
    # Check if each variable exists in the dataset
    for var in variables:
        if var not in ds:
            raise ValueError(f"Variable '{var}' not found in the dataset.")
    
    # Fill NaN values for the specified variables
    filled_ds = ds.copy()
    for var in variables:
        filled_ds[var] = filled_ds[var].fillna(fill_value)
    
    return filled_ds

ds = fillna_in_variables(ds, ["sst"], fill_value=0)

# Drop irrelavat variables 

# Interploation

In [5]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-90, 90.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(0, 360, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if ds.latitude[0] > ds.latitude[-1]:
    ds = ds.sortby("latitude")

# Step 4: Perform interpolation
ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")


In [6]:
ds

<xarray.Dataset> Size: 7GB
Dimensions:    (date: 562, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B 0
  * date       (date) datetime64[ns] 4kB 1978-01-01 1978-02-01 ... 2024-10-01
    expver     (date) <U4 9kB '0001' '0001' '0001' ... '0001' '0005' '0005'
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/23)
    u10        (date, latitude, longitude) float64 293MB 0.3849 0.3849 ... 1.449
    v10        (date, latitude, longitude) float64 293MB 0.002124 ... 1.902
    t2m        (date, latitude, longitude) float64 293MB 242.1 242.1 ... 265.6
    sst        (date, latitude, longitude) float64 293MB 0.0 0.0 ... 271.5 271.5
    sp         (date, latitude, longitude) float64 293MB 6.886e+04 ... 1.01e+05
    tp         (date, latitude, longitude) float64 293MB 6.294e-05 ... 0.001135
    ...         ...
    swvl1      (date, latitude, longitude) float64 293MB -2.484e-06 ... -9.68...
    cvh        (date, latitude, longitude) float64 293MB 0.0 0.0 0.0 ... 0.0 0.0
    lai_hv     (date, latitude, longitude) float64 293MB 0.0 0.0 0.0 ... 0.0 0.0
    tvh        (date, latitude, longitude) float64 293MB 0.0 0.0 0.0 ... 0.0 0.0
    z          (date, latitude, longitude) float64 293MB 2.736e+04 ... -1.24
    lsm        (date, latitude, longitude) float64 293MB 1.0 1.0 1.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

# Smaller data range for testing 

In [7]:
# Filter the dataset for the date range 2016-01-01 to 2023-12-01
ds = ds.sel(date=slice("2016-01-01", "2023-12-01"))


In [8]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B 0
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB '0001' '0001' '0001' ... '0001' '0001' '0001'
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/23)
    u10        (date, latitude, longitude) float64 50MB 0.3059 0.3059 ... 0.1781
    v10        (date, latitude, longitude) float64 50MB -0.1299 ... -0.01796
    t2m        (date, latitude, longitude) float64 50MB 243.4 243.4 ... 252.7
    sst        (date, latitude, longitude) float64 50MB 0.0 0.0 ... 271.5 271.5
    sp         (date, latitude, longitude) float64 50MB 6.878e+04 ... 1.017e+05
    tp         (date, latitude, longitude) float64 50MB 8.774e-05 ... 0.0003529
    ...         ...
    swvl1      (date, latitude, longitude) float64 50MB 0.165 ... -5.417e-06
    cvh        (date, latitude, longitude) float64 50MB 0.0 0.0 0.0 ... 0.0 0.0
    lai_hv     (date, latitude, longitude) float64 50MB 0.0 0.0 0.0 ... 0.0 0.0
    tvh        (date, latitude, longitude) float64 50MB 0.0 0.0 0.0 ... 0.0 0.0
    z          (date, latitude, longitude) float64 50MB 2.736e+04 ... -1.24
    lsm        (date, latitude, longitude) float64 50MB 1.0 1.0 1.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
ds.to_netcdf('/teamspace/studios/this_studio/ml-drought-forecasting/soil-water-forecasting/data/02_intermediate/preprocessed_data.nc')